# Pandas for data cleaning and analysis

```{figure} https://pandas.pydata.org/static/img/pandas_secondary.svg
---
width: 50%
align: center
name: 
---

```


Pandas has two main data structures:
- **`Series`**: A one-dimensional labeled array, like a single column of data.
- **`DataFrame`**: A two-dimensional labeled data structure with columns of potentially different types, similar to a spreadsheet or SQL table.

## The Dataframe
The dataframe is the most important object inside pandas. It allows to represent, access, process, etc multi-dimensional data. 

![Pandas dataframe](https://www.w3resource.com/w3r_images/pandas-data-structure.svg)

Source: https://www.w3resource.com/python-exercises/pandas/index.php

![Pandas dataframe example](https://miro.medium.com/max/1400/1*ZSehcrMtBWN7_qCWq_HiSg.png)

Source: https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c

You can initialize a dataframe in several ways. For example, you can use a dictionary or a nested list. Or you can read from a file, either local or online. 
For example, you can do something like

In [ ]:
# import some needed libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.DataFrame([[909976, "Sweden"],
                   [8615246, "United Kingdom"],
                   [2872086, "Italy"],
                   [2273305, "France"],
                   [344444, np.nan]])
df

In [ ]:
sns.heatmap(df.isnull(), cbar=False)
plt.show()


In [ ]:
import missingno as msno # Check https://github.com/ResidentMario/missingno
%matplotlib inline
msno.matrix(df)

In [ ]:
df.dropna() # returns a new view but does not overwrite the original df

In [ ]:
df.fillna("Unknown")

In [ ]:
df = df.dropna()
df

In [ ]:
df.index = ["Stockholm", "London", "Rome", "Paris"]
df.columns = ["Population", "State"]
df

## Pandas vs. Polars

Polars is a newer, extremely fast DataFrame library built in Rust. It's gaining popularity for its performance, especially on large datasets.

| Feature | Pandas | Polars |
|---|---|---|
| **Backend** | Python/NumPy (partially C) | Rust (built on Apache Arrow)  |
| **Performance** | Slower, especially on large data.  | Significantly faster (5-100x) due to parallelism and query optimization.  |
| **Execution Model** | Eager (executes line-by-line)  | Supports both Eager and Lazy execution (optimizes the whole query before running)  |
| **Memory Usage** | Higher memory footprint.  | More memory efficient.  |
| **API** | Very flexible, but can be inconsistent (e.g., `inplace`). | More consistent and expressive, encourages method chaining.  |
| **Ecosystem** | Mature and extensive. Integrates with almost every data science library (scikit-learn, Matplotlib, etc.). | Growing, but less integrated with the broader ML ecosystem.  |

**When to choose which?**
- **Pandas**: Excellent for data exploration, smaller datasets (up to a few GB), and projects that need deep integration with libraries like scikit-learn.
- **Polars**: Ideal for large datasets, performance-critical data transformations, and building data pipelines where speed and memory are key. 

### Polars Syntax Example

Notice the similarity, but also the use of expressions (`pl.col()`).

**Simple exercise**
Complete the demo (find a data source) and create a script with inline dependencies. 
```python
import polars as pl

# Same data in a Polars DataFrame
df_pl = pl.DataFrame(data)

# The same aggregation, but using the Polars expression API
polar_stats = df_pl.group_by('Experiment').agg(
    pl.col('Measurement').mean().alias('mean'),
    pl.col('Measurement').std().alias('std')
)

print(polar_stats)
```
:::

## Applied pandas tutorial

In this tutorial we will explore a real data escenario to look for exoplanets! We will use real data and `pandas` to clean it and get some useful info. The main goal is to be able to answer questions like:
- What is the most common method for discovering exoplanets?
- What is the average size of a planet discovered in the last decade?
- Is there a relationship between a star's temperature and the mass of its planets?

But when you first look at the data, you realize it's not that simple. There are gaps, inconsistencies, and errors. This is where we need to clean the data.

### Getting the data
First, let's pull the data directly from the NASA Exoplanet Archive. This is real, live data! <https://exoplanetarchive.ipac.caltech.edu/docs/data.html>

The columns we are requesting are:

    pl_name: Planet Name

    hostname: Host Star Name

    discoverymethod: How the planet was found

    pl_orbper: Orbital Period (days)

    pl_rade: Planet Radius (Earth radii)

    pl_bmasse: Planet Mass (Earth masses)

    st_teff: Star's Effective Temperature (Kelvin)

    st_rad: Star's Radius (Solar radii)

    st_mass: Star's Mass (Solar masses)

    disc_year: Discovery Year



In [ ]:
import pandas as pd
import numpy as np # We'll need numpy for more advanced NaN handling

# The official URL from the NASA Exoplanet Archive
# https://exoplanetarchive.ipac.caltech.edu/docs/data.html
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync?query=select+pl_name,hostname,discoverymethod,pl_orbper,pl_rade,pl_bmasse,st_teff,st_rad,st_mass,disc_year+from+ps&format=csv"

# Load the data into a pandas DataFrame
df = pd.read_csv(url) # go and read the manual

In [ ]:
print("Our Raw Exoplanet Data (First 5 Rows):")
print(df.head())

In [ ]:
print("\n--- Mission Critical Info ---")
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull()

Analyze the output:
- `Total entries`
- `Non-null count` : Significant missing data problem
- `Dtype`: getting `float` or `int` is good (where it makes sense). If all were objects, that would signal hidden text or errors.

Before doing any chance, let's quantify the missing data.

:::{exercise} Quantifying the void
Use a single pandas command to calculate the total number of missing values for each column. What column has the most missing data? does it make sense (measuring mass or radius can be much harder that just measuring the presence)
:::

In [ ]:
# YOUR CODE HERE



You can also access columns or groups of columns like


In [ ]:
df[['pl_orbper', 'pl_rade']]

### Handling missing values
Missing data will affect your analysis. To handle it, you need to think _why_ the data is missing, and _how much_ data is missing.

Let's focus on the key planetary characteristics: pl_orbper, pl_rade, and pl_bmasse.

Dropping all rows with any missing data (`df.dropna()`) would be a disaster: we'd lose a huge portion of our catalog! A better approach is imputation: filling the gaps with a sensible calculated value.

The *median* is often a better choice for imputation than the *mean* for astronomical data, as it's less sensitive to extreme outliers (e.g., a planet with a gigantic radius or a very long orbital period). Check also https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html .



In [ ]:
# It's good practice to work on a copy
df_cleaned = df.copy()

# Calculate the median for the columns we want to fill
# We'll calculate them from the original data before we start changing it
median_radius = df_cleaned['pl_rade'].median()
median_mass = df_cleaned['pl_bmasse'].median()
median_orb_period = df_cleaned['pl_orbper'].median()

# Impute the missing values using the calculated medians
# df_cleaned['pl_rade'].fillna(median_radius, inplace=True) # Avoid inplace=True in a copy object
df_cleaned['pl_rade'] = df_cleaned['pl_rade'].fillna(median_radius)
df_cleaned['pl_bmasse'] = df_cleaned['pl_bmasse'].fillna(median_mass)
df_cleaned['pl_orbper'] = df_cleaned['pl_orbper'].fillna(median_orb_period)
# # or better use
# df_cleaned.fillna({
#     'pl_rade': df_cleaned['pl_rade'].median(),
#     'pl_bmasse': df_cleaned['pl_bmasse'].median(),
#     'pl_orbper': df_cleaned['pl_orbper'].median()
# }, inplace=True)

In [ ]:
print("--- Missing Values After Imputation ---")
print(df_cleaned.isnull().sum())

Now our core planet columns are filled! We still have missing data in the star properties (st_teff, st_rad, st_mass), but we can leave them for now, as our analysis might not always require them.

:::{exercise} Missing star data
1. Choose one of the star-related columns with missing data (st_teff, st_rad, or st_mass).
2. Calculate the median for that column.
3. Impute (fill) the missing values in that column using its median.
4. Verify your work by checking the isnull().sum() count for that column again. It should be zero!
:::


In [ ]:
# YOUR CODE HERE



### Correcting Cosmic Anomalies (Outliers)
Outliers are extreme values that can skew our understanding. They could be real, fascinating discoveries (like a planet with a massive orbit) or simple data entry errors. Visualizing the data is the best way to spot them.

A **box plot** is a powerful tool for outlier detection.




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set a nice style for our plots
sns.set_theme(style="whitegrid")

# Let's visualize the distribution of Planet Radius
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_cleaned['pl_rade'])
plt.title('Distribution of Exoplanet Radii (in Earth Radii)')
plt.xscale('log') # Log scale is often essential for astronomical data!
plt.show()

In [ ]:
# Plot showing data
# Maybe use stripplot, swarmplot, or even a violin plot
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="darkgrid")
plt.figure(figsize=(10, 6))

# First the boxplot
sns.boxplot(x=df_cleaned['pl_rade'])

# Overlay the data as scatter (stripplot)
sns.stripplot(x=df_cleaned['pl_rade'], 
              color='brown', 
              alpha=0.2,   # transparency
              size=3,      # point size
              jitter=True) # adds random jitter to avoid overlap

plt.title('Distribution of Exoplanet Radii (in Earth Radii)')
plt.xscale('log')
plt.xlabel('pl_rade')
plt.show()


Notice that the box plot shows many points far to the right. These are planets with very large radii. A log scale helps, but we can see that most planets are clustered in a smaller range. Let's say, for a specific analysis, we want to focus only on planets that are somewhat similar to Earth, and we consider anything over 20 Earth radii to be a "super-giant" that we want to handle separately.

This is not about deleting data, but about filtering it for a specific analysis.

In [ ]:
# How many planets are larger than 20 Earth radii?
super_giants = df_cleaned[df_cleaned['pl_rade'] > 20] # you can use any other condition
print(f"There are {len(super_giants)} planets with a radius greater than 20 Earths.")

# For our analysis, let's create a new DataFrame without these giants
df_filtered = df_cleaned[df_cleaned['pl_rade'] <= 20].copy()

print(f"\nOriginal DataFrame size: {len(df_cleaned)}")
print(f"Filtered DataFrame size: {len(df_filtered)}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set a nice style for our plots
sns.set_theme(style="whitegrid")

# Let's visualize the distribution of Planet Radius
plt.figure(figsize=(10, 6))
sns.boxplot(x=df_filtered['pl_rade'])
plt.title('Distribution of Exoplanet Radii (in Earth Radii) - filtered')
plt.xscale('log') # Log scale is often essential for astronomical data!
plt.show()

:::{exercise} Explain this command

:::

In [ ]:
# Explain this syntax
tmp = df_cleaned[df_cleaned['pl_rade'] > 20].iloc[100:110] # access specific rows
tmp

:::{exercise} Planet mass
1. Create a box plot for pl_bmasse. Remember to use a log scale (plt.xscale('log')) to get a better view.

2. Based on the plot, choose a reasonable upper limit to filter out extreme outliers (for example, you might decide anything over 3000 Earth masses is an outlier worth investigating).

3. Filter the df_filtered DataFrame further to remove these mass outliers. Print the number of rows before and after to see how many planets you filtered.
:::

In [ ]:
# YOUR CODE HERE



### More analysis
Now let's try to answer the following question: What is the most common discovery method for each decade?

To answer this, we need to make sure our `discoverymethod` and `disc_year` columns are clean and ready.

In [ ]:
# Let's check the different discovery methods
print("Unique Discovery Methods:")
print(df_filtered['discoverymethod'].value_counts())

The data looks clean. Now, let's use our cleaned data to answer the question.

In [ ]:
# Let's create a 'decade' column for easier grouping
# We can do this by integer division
df_filtered['discovery_decade'] = (df_filtered['disc_year'] // 10) * 10
df_filtered.head()

In [ ]:
# Now, let's count the methods per decade
discovery_summary = df_filtered.groupby('discovery_decade')['discoverymethod'].value_counts()

In [ ]:
print("\n--- Discovery Method Counts per Decade ---")
print(discovery_summary)

:::{exercise} Visualizing the discovery data
1. The discovery_summary data is a Series. To plot it effectively with seaborn, it's often easier to convert it into a DataFrame. Use the .reset_index() method on discovery_summary.

2. Rename the new column (which will likely be named count or 0) to something more descriptive, like planet_count.

3. Create a bar plot using seaborn.barplot() to show the planet_count for each discoverymethod, grouped by discovery_decade. Hint: Use x='discovery_decade', y='planet_count', and hue='discoverymethod'.

4. Give your plot a good title, like "Dominant Exoplanet Discovery Methods by Decade".
:::

In [ ]:
# YOUR CODE HERE



What can you conclude from your plot? Which method dominated the 2010s?

### More about pandas
`Pandas` offers much more than what was shown here. Please check:
- `apply`: to apply a function to a column. Example: make all pl_name lower: `df['pl_name'].apply(lambda x: x.lower()) `
- Operation across columns: `df['what is this'] = df['pl_rade'] + df['pl_orbper']`
- Concatenation: `concat`
- Merge: `merge`
- `query`: `df.query('A > B')`, <https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html>
- `at`: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.at.html
- `where`: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.where.html
- ...


## Future Learning


### Polars
```{figure} https://pypi-camo.freetls.fastly.net/fac9f1945cd7594df0567bc53f726274340624f5/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f706f6c612d72732f706f6c6172732d7374617469632f6d61737465722f62616e6e65722f706f6c6172735f6769746875625f62616e6e65722e737667
---
width: 50%
align: center
name: 
---

```


[Polars](https://pola.rs/) is a newer, extremely fast DataFrame library built in Rust. It's gaining popularity for its performance, especially on large datasets. See the [polars documentation](https://docs.pola.rs/) . It seems that the current pandas advantage over polars is [geopandas](https://geopandas.org/en/stable/). 

| Feature | Pandas | Polars |
|---|---|---|
| **Backend** | Python/NumPy (partially C) | Rust (built on Apache Arrow)  |
| **Performance** | Slower, especially on large data.  | Significantly faster (5-100x) due to parallelism and query optimization.  |
| **Execution Model** | Eager (executes line-by-line)  | Supports both Eager and Lazy execution (optimizes the whole query before running)  |
| **Memory Usage** | Higher memory footprint.  | More memory efficient.  |
| **API** | Very flexible, but can be inconsistent (e.g., `inplace`). | More consistent and expressive, encourages method chaining.  |
| **Ecosystem** | Mature and extensive. Integrates with almost every data science library (scikit-learn, Matplotlib, etc.). | Growing, but less integrated with the broader ML ecosystem.  |


```{figure} https://pola.rs/_astro/perf-illustration.jHjw6PiD_hpOso.svg
---
width: 50%
align: center
name: 
---

```


**When to choose which?**
- **Pandas**: Excellent for data exploration, smaller datasets (up to a few GB), and projects that need deep integration with libraries like scikit-learn.
- **Polars**: Ideal for large datasets, performance-critical data transformations, and building data pipelines where speed and memory are key. 

#### Polars Syntax Example

Notice the similarity, but also the use of expressions (`pl.col()`).

**Simple exercise**
Complete the demo (find a data source) and create a script with inline dependencies. 

```python
import polars as pl

# Same data in a Polars DataFrame
df_pl = pl.DataFrame(data)

# The same aggregation, but using the Polars expression API
polar_stats = df_pl.group_by('Experiment').agg(
    pl.col('Measurement').mean().alias('mean'),
    pl.col('Measurement').std().alias('std')
)

print(polar_stats)
```
